In [ ]:
# Importing tensorflow to the python shell!
import tensorflow as tf

In [ ]:
# Defining a node of a constant in tensorflow 
# tf.constant returns a tf.Tensor and defines a tf.Operation in the graph 
a = tf.constant(42,dtype=tf.int32,name='my_a',shape=(1,))
b = tf.constant(42,dtype=tf.int32,name='my_b',shape=(1,))

In [ ]:
# Defining a sum in tensorflow
# will add a tf.Operation to the graph
c = tf.add(a,b) 
# or c = a + b, tensorflow accepts a range of arithmetic and logical ops

In [ ]:
# but what is c? c is still a tensor, tf.add() returns a tensor and
# adds an operation to the graph
c

In [ ]:
# we need to create a session in order to run a specific node in the
# graph. Furthermore, we have been adding nodes to the "default" graph,
# created when importing the tensorflow module, but we could've created
# a different one

In [ ]:
# Create a session object to run nodes in the graph
sess = tf.Session()

In [ ]:
# Now we may run any node in the graph by using the tf.Session.run()
# on any node we wish. 
# The nodes are the tensors and the operations the edges of the graph!

In [ ]:
print('My value of a is {}, b is {} and the sum c is {}'.format(sess.run(a),sess.run(b),sess.run(c)))

In [ ]:
# We may close a session with sess.close() which will give us an error when trying to run a node
sess.close()
sess.run(a)

In [ ]:
# We can open a different session with tf.Session() again.
sess = tf.Session()

In [ ]:
# However, we need to dive deeper in the tensorflow structure. Tensorflow was designed to have superior computational
# efficienty at a cost of having to define a computational graph and associated variables properly
# Let's see how many tensors and operations we have so far

In [ ]:
# Get all the tensors currently in our default graph
tf.contrib.graph_editor.get_tensors(tf.get_default_graph())

In [ ]:
# And visualize each tf.Operation added to the graph
sess.graph.get_operations()

In [ ]:
# We may notice that we have the same amount of tf.Tensors and tf.Operations.
# The fact is, when creating anything in TensorFlow we always need a space in memory to physically have each tensor
# And we also need a way to compute and feed each tensor onto other nodes, by defining the operations (edges)
# Let's save our graph and visualize it on tensorboard!
tf.train.write_graph(tf.get_default_graph(), 'folder', 'our_graph.pb')

In [ ]:
# However, we may reset the graph and all the nodes we created, since jupyter notebook is an interactive
# python shell, remenber to reset the graph from time to time :>
tf.reset_default_graph()

In [ ]:
# And visualize there are no tensors or operations in our graph
tf.contrib.graph_editor.get_tensors(tf.get_default_graph())

In [ ]:
# One of the important things to be adressed. Everytime we call a tf function
# we create a tensor (and consequently an operation), this leads to subtle memory leaks.
for i in range(5):
    a = tf.constant(42,dtype=tf.int32,name='my_a',shape=(1,))
tf.contrib.graph_editor.get_tensors(tf.get_default_graph())

In [ ]:
tf.reset_default_graph()
# When desiging graphs (such as neural networks) it is a good practice to define all the nodes at start
# and running only the nodes in our code
a = tf.constant(42,dtype=tf.int32,name='my_a',shape=(1,))
for i in range(5):
    print(sess.run(a)) 
tf.contrib.graph_editor.get_tensors(tf.get_default_graph())

In [ ]:
# Handy Reset
tf.reset_default_graph()

In [ ]:
# We may create tf.Variables, which will hold a variable Tensor during computations
# Most nodes will be composed of tf.Variables.
# There are two ways to define new variables in TensorFlow, by creating a 
# tf.Variable object or by calling tf.get_variable. 
# Calling tf.get_variable with a new name results in creating a new variable, 
# but if a variable with the same name exists it 
# will raise a ValueError exception, telling us that re-declaring 
# a variable is not allowed.
d = tf.get_variable('my_d',[1,2,3]) #Here we already defined the shape 1-D array
e = tf.get_variable('my_e', shape=(18,)) #Here the shape an 1-D array
f = tf.get_variable('my_f', dtype=tf.int32, initializer=tf.constant([23, 42])) 
#Here we defined the type and the initializer which happens to be a constant tf.constant, the shape is defined
tf.contrib.graph_editor.get_tensors(tf.get_default_graph())
# Why are there so many tensors? the second argument is the shape. The default initializer of variables
# is a random uniform distribution. Each Tensor of this Initializer is also created under the name of each variable
# Notice the Tensors Assign and read, serve to feed and fetch values from the Tensor 'my_variable0':0
# In the 'my_f' variable we only have 4 tensors, the initializer, which is a tf.constant, the "de facto" variable
# 'my_f:0' and 2 other tensors to read and assing values to 'my_f:0'

In [ ]:
# Lets get the session again, instead of creating a new tf.Session, this is handy on long, complex programs.
# If there isn't any tf.Session available, it will return 'None', so we need to create another session.
# sess = tf.Session()
sess = tf.get_default_session()
# Run the node f and print it
sess.run(f)
# Why did we got an error? the tf.initalizer is a method, and has not been run, therefore
# there are no values to be retrieved when we run the node. We have two options: 
# 1) Either run a particular variable initializer with sess.run(f.initializer)
# 2) Or run the tf.global_variables_initializer() which will initialize all the nodes created in the graph thus far

In [ ]:
sess.run(tf.global_variables_initializer())
# Every node (tensor) when runned, return an numpy array
sess.run(f)

In [ ]:
tf.reset_default_graph()
# So far we have seen tf.constant and tf.get_variable. However there is also a data structure appropriate to 
# feed data into our models. We could feed them through variables, however variables are more heavy and have
# initializers, read, write tensors... it is a good practice to use tf.placeholders on entry nodes of our model
# since they are lightweight and specifically suited for this task
g = tf.placeholder(tf.float32, shape=(1,),name='entrada')
tf.contrib.graph_editor.get_tensors(tf.get_default_graph())
# As we can see, placeholders are very light weight, holding only 1 tensor
# using them to feed data into our model is extremely clean!

In [48]:
tf.reset_default_graph()
# How do we run a specific node while feeding data at the same time?
# On of the arguments of tf.Session.run(node,feed_dict) is feed_dict
# which is a python dictionary (also obeys the same sintax of json).
# Thus, we run a node, while feeding certain dependant nodes.
# For example, let us run a sum with a variable and a placeholder
x_1 = tf.placeholder(tf.int32, (),'x_1')
x_2 = tf.get_variable('x_2', dtype=tf.int32 ,initializer=tf.constant(21))
x_result = x_1 + x_2 #this adition will make one tensor and one operation
tf.contrib.graph_editor.get_tensors(tf.get_default_graph())

[<tf.Tensor 'x_1:0' shape=() dtype=int32>,
 <tf.Tensor 'Const:0' shape=() dtype=int32>,
 <tf.Tensor 'x_2:0' shape=() dtype=int32_ref>,
 <tf.Tensor 'x_2/Assign:0' shape=() dtype=int32_ref>,
 <tf.Tensor 'x_2/read:0' shape=() dtype=int32>,
 <tf.Tensor 'add:0' shape=() dtype=int32>]

In [50]:
# Now lets run the node x_result with a custom x_1, after initializing
# x_2!
sess = tf.Session()
# sess = tf.get_default_session()
sess.run(x_2.initializer)
# Create a dictionary to feed x_1
dictionary = {x_1 : 21}
# Run x_result node, while 
sess.run(x_result,feed_dict=dictionary)

42

In [35]:
sess.run(x_2)

21

In [52]:
# We can, however, feed variables aswell (placeholder always need to be fed)
sess.run(x_result,feed_dict={x_1:21,x_2:21})
# Another important note: 
# shape=() is a single number 0-D
# shape = (1,) is a 1-D, usual python list
# shape = (1,1) is a 2-D, a python list of a list with a single point

42

In [106]:
# We may group variables, placeholders and constants into a 
# name scope. This is useful when we want to handle a group of 
# variables, weights at the same time, such as initializing them
# with the same distribution, or saving them into a particular place.
# It also makes the graph well organized and easier to observe
# and debug in tensorboard :>
# Lets create 2 scopes, one for sum and one for subtraction
tf.reset_default_graph()
with tf.variable_scope('sum') as scope:
    x_1 = tf.get_variable('x_1',dtype=tf.int32,initializer=tf.constant(21))
    x_2 = tf.get_variable('x_2',dtype=tf.int32,initializer=tf.constant(21))
    x_sum = tf.add(x_1,x_2) #or x_1 + x_2

with tf.variable_scope('subtract'):
    x_1 = tf.get_variable('x_1',dtype=tf.int32,initializer=tf.constant(84))
    x_2 = tf.get_variable('x_2',dtype=tf.int32,initializer=tf.constant(42))
    x_subtract = tf.subtract(x_1,x_2) #or x_1 - x_2
tf.contrib.graph_editor.get_tensors(tf.get_default_graph())
# As we can see we have 2 sets of tensors, one grouped under the name 'sum' 
# and another one grouped at 'subtract'. Furthermore, one might say we have
# in TensorFlow language, 2 name_scopes each one with a collection of tensors

[<tf.Tensor 'sum/Const:0' shape=() dtype=int32>,
 <tf.Tensor 'sum/x_1:0' shape=() dtype=int32_ref>,
 <tf.Tensor 'sum/x_1/Assign:0' shape=() dtype=int32_ref>,
 <tf.Tensor 'sum/x_1/read:0' shape=() dtype=int32>,
 <tf.Tensor 'sum/Const_1:0' shape=() dtype=int32>,
 <tf.Tensor 'sum/x_2:0' shape=() dtype=int32_ref>,
 <tf.Tensor 'sum/x_2/Assign:0' shape=() dtype=int32_ref>,
 <tf.Tensor 'sum/x_2/read:0' shape=() dtype=int32>,
 <tf.Tensor 'sum/Add:0' shape=() dtype=int32>,
 <tf.Tensor 'subtract/Const:0' shape=() dtype=int32>,
 <tf.Tensor 'subtract/x_1:0' shape=() dtype=int32_ref>,
 <tf.Tensor 'subtract/x_1/Assign:0' shape=() dtype=int32_ref>,
 <tf.Tensor 'subtract/x_1/read:0' shape=() dtype=int32>,
 <tf.Tensor 'subtract/Const_1:0' shape=() dtype=int32>,
 <tf.Tensor 'subtract/x_2:0' shape=() dtype=int32_ref>,
 <tf.Tensor 'subtract/x_2/Assign:0' shape=() dtype=int32_ref>,
 <tf.Tensor 'subtract/x_2/read:0' shape=() dtype=int32>,
 <tf.Tensor 'subtract/Sub:0' shape=() dtype=int32>]

In [107]:
# Running the 2 operations is easy. We initialize variables and run the result nodes
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print('The sum is {} and the subtraction is {}'.format(sess.run(x_sum),sess.run(x_subtract)))

The sum is 42 and the subtraction is 42


In [76]:
# We may only look to one set of variables from one specific selection,
# tf.Variables hold tensors+operations, we may select a set to reuse,
# or to save. 
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

[<tf.Variable 'sum/x_1:0' shape=() dtype=int32_ref>,
 <tf.Variable 'sum/x_2:0' shape=() dtype=int32_ref>,
 <tf.Variable 'subtract/x_1:0' shape=() dtype=int32_ref>,
 <tf.Variable 'subtract/x_2:0' shape=() dtype=int32_ref>]

In [77]:
# Or under a specific scope
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope='sum')

[<tf.Variable 'sum/x_1:0' shape=() dtype=int32_ref>,
 <tf.Variable 'sum/x_2:0' shape=() dtype=int32_ref>]

In [116]:
path=!pwd
path

['/home/daniel/TensorFlow-Tutorials']

In [120]:
# We may save our session, to continue it later. This will freeze every tensor
# assigned inner value and save in an appropriate file to be restored.
saver = tf.train.Saver()
path = !pwd # path to this folder on your computer, ! computes in bash
saver.save(sess,path[0]+"/model/model")

'/home/daniel/TensorFlow-Tutorials/model/model'

In [ ]:
saver.restore(sess,)

In [108]:
# For instance, we may use all variables from a scope
sums = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope='sum') #List of Variables
# For every tf.Variable, we will add 1
sumcity = []
for var in sums:
    sumcity.append(var + 1)

sumscity = tf.group(*sumcity)    
#print(sess.run(var))
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope='sum')
print('sums has {} objects and x_sum is {} '.format(len(sums),sess.run(x_sum)))
tf.contrib.graph_editor.get_tensors(tf.get_default_graph())
# If the graph is leaking because you are running this several times, reset it

sums has 2 objects and x_sum is 42 


[<tf.Tensor 'sum/Const:0' shape=() dtype=int32>,
 <tf.Tensor 'sum/x_1:0' shape=() dtype=int32_ref>,
 <tf.Tensor 'sum/x_1/Assign:0' shape=() dtype=int32_ref>,
 <tf.Tensor 'sum/x_1/read:0' shape=() dtype=int32>,
 <tf.Tensor 'sum/Const_1:0' shape=() dtype=int32>,
 <tf.Tensor 'sum/x_2:0' shape=() dtype=int32_ref>,
 <tf.Tensor 'sum/x_2/Assign:0' shape=() dtype=int32_ref>,
 <tf.Tensor 'sum/x_2/read:0' shape=() dtype=int32>,
 <tf.Tensor 'sum/Add:0' shape=() dtype=int32>,
 <tf.Tensor 'subtract/Const:0' shape=() dtype=int32>,
 <tf.Tensor 'subtract/x_1:0' shape=() dtype=int32_ref>,
 <tf.Tensor 'subtract/x_1/Assign:0' shape=() dtype=int32_ref>,
 <tf.Tensor 'subtract/x_1/read:0' shape=() dtype=int32>,
 <tf.Tensor 'subtract/Const_1:0' shape=() dtype=int32>,
 <tf.Tensor 'subtract/x_2:0' shape=() dtype=int32_ref>,
 <tf.Tensor 'subtract/x_2/Assign:0' shape=() dtype=int32_ref>,
 <tf.Tensor 'subtract/x_2/read:0' shape=() dtype=int32>,
 <tf.Tensor 'subtract/Sub:0' shape=() dtype=int32>,
 <tf.Tensor 'add/

In [110]:
sess.run(sumcity)

[22, 22]

In [ ]:
sess.graph.get_tensor_by_name('my_a:0')

tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

g = tf.get_variable('my_d') #Here we use the same tensor name 'my_d' created in d to perform
# a different computation with the same weights on another part of the program

When we run a particular node,  

sess.run(a)

#feed and run, save and restore models

session.run(b)

sess.close()